In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "parser"

In [2]:
from typing import Literal
from langchain_core.pydantic_v1 import BaseModel, Field


class ParsedResult(BaseModel):
    action: str = Field(description="The action that can be executed")
    interact_with: str = Field(description="The contract address to interact with")


class ActionTransfer(ParsedResult):
    action: Literal["transfer"] = Field(description="The action to be taken")
    amount: float = Field(description="The amount to be transferred")
    currency: str = Field(description="The currency of the transfer")
    recipient: str = Field(description="The recipient of the transfer")


class ActionSwap(ParsedResult):
    action: Literal["swap"] = Field(description="The action to be taken")
    description: str = Field(description="The description of the swap")

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from utils.model_selector import get_chat_model

system_prompt = (
    "You are a Ethereum expert that can extract the action from a description."
)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{input}")]
)

llm = get_chat_model().model


def parse_description(description: str) -> ParsedResult:
    llm_with_output = llm.with_structured_output(ParsedResult)
    chain = prompt | llm_with_output
    return chain.invoke({"input": description})

In [4]:
print(parse_description("Swap 100 USDC to ETH"))
print(parse_description("Approve 100 USDC to be swapped"))
print(parse_description("Transfer 100 USDC to Alice"))

action='swap' interact_with='0xUniswapV2RouterAddress'
action='approve' interact_with='USDC_contract_address'
action='transfer' interact_with='USDC_contract_address'


In [9]:
supported_actions = [
    "transfer",
    "swap",
    "approve",
    "mint",
    "burn",
    "stake",
    "bridge",
    "repay",
    "withdraw",
    "borrow",
    "deposit",
]
from langchain_core.runnables import Runnable, RunnableLambda


def parse_description(result: ParsedResult) -> bool:
    if action := result.action in supported_actions:
        return True
    else:
        raise ValueError(f"Action {action} is not supported")


chain = prompt | llm.with_structured_output(ParsedResult) | parse_description
chain.invoke({"input": "Swap 100 USDC to ETH"})

True

In [ ]:
from langgraph.prebuilt import create_react_agent